<a href="https://colab.research.google.com/github/clairemasse/Coursera_capstone/blob/master/Comparing_Neighborhoods_in_London_%26_New_York_City.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Coursera Capstone Final Project:


In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np
from IPython.display import display
import requests
import json
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
# transforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed!')

/bin/bash: conda: command not found
Folium installed!


## Introduction/Business Problem: ##

I lived in New York City for  3 years, the last of which I lived in Bedford-Stuyvesant, Brooklyn. I am hoping to move to London in the next few months, and will need to think about which neighborhood I should aim to live in. So for this project, I decided to compare neighborhoods in both cities to see which neighborhood in London is the most similar to the one I lived in in Brooklyn. 

I will be using geographical data available online for both cities as well as the Foursquare API in order to examine and compare the venues (restaurants, bars, gyms, stores...) in different neighborhoods of both cities. 

In [2]:
# Importing Brooklyn neighborhood data
bk = pd.read_html('http://www.brooklynproperty.com/Brooklyn/ZipCodes.htm')[0]

In [3]:
bk

,City Name,Zip Code,City Name.1,Zip Code.1
0,Brooklyn Heights/Cobble Hill,11201,East Flatbush,11203.0
1,Parkville/Bensonhurst,11204,Fort Greene,11205.0
2,Williamsburg/Bedford-Stuyvesant,11206,East New York,11207.0
3,Cypress Hills,11208,Bay Ridge,11209.0
4,Vanderveer/Flatbush,11210,Williamsburg,11211.0
5,Brownsville,11212,Brower Park/Crown Heights,11213.0
6,Bath Beach/Bensonhurst,11214,Park Slope/Windsor Terrace,11215.0
7,Bedford-Stuyvesant,11216,Park Slope/Gowanus,11217.0
8,Kensington/Windsor Terrace,11218,Borough Park,11219.0
9,Sunset Park,11220,Bushwick/Bedford-Stuyvesant,11221.0


In [4]:
#Clean up the data so it's all in one data frame with 2 columns, neighborhood name and zip code

bk_1 = bk.filter(['City Name', 'Zip Code'], axis = 1)
bk_2 = bk.filter(['City Name.1', 'Zip Code.1'], axis = 1)

bk_2.rename(columns = {'City Name.1': 'City Name', 'Zip Code.1': 'Zip Code'}, inplace=True)

bk = bk_1.append(bk_2)

bk.reset_index(inplace=True)
bk.drop(columns = 'index', inplace=True)

bk.dropna(inplace=True)

bk['Zip Code'] = bk['Zip Code'].astype(int)

bk.rename(columns = {'City Name':'Neighborhood', 'Zip Code': 'Post Code'}, inplace= True)
bk.head()

,Neighborhood,Post Code
0,Brooklyn Heights/Cobble Hill,11201
1,Parkville/Bensonhurst,11204
2,Williamsburg/Bedford-Stuyvesant,11206
3,Cypress Hills,11208
4,Vanderveer/Flatbush,11210


In [5]:
# Define lat/lon functions to add columns to dataframe

geolocator = Nominatim(user_agent="foursquare_agent")

def latitude(address):
  location = geolocator.geocode(address)
  lat = location.latitude
  return lat

def longitude(address):
  location = geolocator.geocode(address)
  lon = location.longitude
  return lon


In [6]:
bk['latitude'] = bk['Post Code'].apply(lambda x: latitude(x))
bk['longitude'] = bk['Post Code'].apply(lambda x: longitude(x))

In [7]:
bk

,Neighborhood,Post Code,latitude,longitude
0,Brooklyn Heights/Cobble Hill,11201,40.696131,-73.996413
1,Parkville/Bensonhurst,11204,40.617336,-73.985333
2,Williamsburg/Bedford-Stuyvesant,11206,40.697646,-73.942416
3,Cypress Hills,11208,40.674337,-73.865342
4,Vanderveer/Flatbush,11210,43.022532,3.035715
5,Brownsville,11212,54.689959,25.421529
6,Bath Beach/Bensonhurst,11214,40.596492,-73.999159
7,Bedford-Stuyvesant,11216,54.686545,25.424749
8,Kensington/Windsor Terrace,11218,54.688060,25.426231
9,Sunset Park,11220,43.091901,2.620856


In [8]:
ldn = pd.read_csv('https://raw.githubusercontent.com/clairemasse/Coursera_capstone/master/london_postcodes%20-%20London_postcodes%20-%20Sheet2%20(1)%20(1).csv')

In [ ]:
ldn['location'] = ldn['Neighborhood'].apply(lambda x: geolocator.geocode(x))
ldn.dropna(inplace=True)

In [ ]:
ldn.reset_index(inplace=True)
ldn

In [ ]:
ldn.drop(['index', 'location'],axis=1,inplace=True)

In [ ]:
ldn['latitude'] = ldn['Neighborhood'].apply(lambda x:latitude(x))
ldn['longitude'] = ldn['Neighborhood'].apply(lambda x: longitude(x))

In [ ]:
ldn

In [ ]:
neighborhoods = ldn.append(bk)

In [ ]:
neighborhoods.reset_index(inplace=True)


In [ ]:
neighborhoods.drop('index', axis=1, inplace= True)

In [ ]:
neighborhoods

In [ ]:
# create map of Brooklyn using latitude and longitude values
bk_lat = 40.650002
bk_lon = -73.949997
map_bk = folium.Map(location=[bk_lat, bk_lon], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bk)  
    
map_bk

In [ ]:
# create map of London using latitude and longitude values
ldn_lat = 51.509865
ldn_lon = -0.118092
map_ldn = folium.Map(location=[ldn_lat, ldn_lon], zoom_start=11.5)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ldn)  
    
map_ldn

In [ ]:
# Now that we have all the neighborhoods in both cities marked and identified, it's time cluster them using Foursquare API location data

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
# Foursquare API credentials

CLIENT_ID = 'GLELENDKNLLSRPHH3U1MA2UMELUP5XGF2BLSAPNBONU1JNUN' 
CLIENT_SECRET = 'UUMVCYSAIG5SXZR2KAD1WIROGCP5DIEM3N3E4BT1I1QEDSAE' 
VERSION = '20180605' 



In [ ]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
bkldn_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['latitude'],
                                   longitudes=neighborhoods['longitude']
                                  )

In [ ]:
print('The dataframe returns', bkldn_venues.shape[0], 'venues.')
bkldn_venues.head()

In [ ]:
# Number of venues per neighborhood
display(bkldn_venues.groupby('Neighborhood')['Venue'].count().head(10))

In [ ]:
print('There are {} unique categories.'.format(len(bkldn_venues['Venue Category'].unique())))

We will use one-hot encoding to analyze all the different neighborhoods

In [ ]:
# one hot encoding
bkldn_onehot = pd.get_dummies(bkldn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bkldn_onehot['Neighborhood'] = bkldn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bkldn_onehot.columns[-1]] + list(bkldn_onehot.columns[:-1])
bkldn_onehot = bkldn_onehot[fixed_columns]

bkldn_onehot.head()

In [ ]:
#Let's group by neighborhood and take the mean of the frequency of occurence of each category

bkldn_grouped = bkldn_onehot.groupby('Neighborhood').mean().reset_index()
display(bkldn_grouped.head())


In [ ]:
#Top 5 venues in each neighborhood

num_top_venues = 5

for hood in bkldn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bkldn_grouped[bkldn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
### Put above into a pandas dataframe

# Function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]


# create dataframe and display top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bkldn_grouped['Neighborhood']

for ind in np.arange(bkldn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bkldn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

CLUSTER USING K-MEANS


In [ ]:
# set number of clusters
kclusters = 4

bkldn_grouped_clustering = bkldn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bkldn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bkldn_merged = neighborhoods

# merge bkldn data with original data to add latitude/longitude for each neighborhood
bkldn_merged = bkldn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
bkldn_merged.head() 

In [ ]:
bkldn_merged.groupby('Cluster Labels').count()